In [14]:
import torch
import numpy as np
from torch import nn

In [3]:
layer1 = nn.Linear(8,3)

In [4]:
torch.ones((3,8))

tensor([[1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [5]:
layer1.weight.size(1)

8

In [6]:
layer1.weight[:,4:], layer1.weight[:,:4]

(tensor([[-0.0581,  0.0361, -0.0649,  0.0933],
         [ 0.2731, -0.2574, -0.2784, -0.2805],
         [ 0.2419,  0.2746,  0.2415,  0.3205]], grad_fn=<SliceBackward0>),
 tensor([[-0.3356, -0.2775,  0.2027,  0.0922],
         [-0.1183,  0.1392, -0.2094,  0.2146],
         [-0.0479,  0.0785,  0.1698, -0.1927]], grad_fn=<SliceBackward0>))

In [7]:
x = torch.ones((2,3))
y = torch.zeros((2,6))
z = torch.cat((x,y),dim=1)
z

tensor([[1., 1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0., 0.]])

In [8]:
x = torch.ones((18,20))
gama, beta = torch.split(x,10,1)
gama.shape, beta.shape

(torch.Size([18, 10]), torch.Size([18, 10]))

In [36]:
from transformers import AutoModelForSeq2SeqLM
from transformers import BertModel, BertConfig
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType

bertconfig = BertConfig.from_pretrained('bert-base-uncased', output_hidden_states=True)

peft_config = LoraConfig(
    task_type=TaskType.TOKEN_CLS, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1
)

model = BertModel.from_pretrained('bert-base-uncased', config=bertconfig)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()
# trainable params: 294912 || all params: 109777152 || trainable%: 0.2686460658042941

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


trainable params: 294912 || all params: 109777152 || trainable%: 0.2686460658042941


In [42]:
from transformers import BertTokenizer

bertTokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
inputs = bertTokenizer.encode_plus("Tweet text : @HondaCustSvc Your customer service has been horrible during the recall process. I will never purchase a Honda again. Label :", return_tensors="pt",max_length=512,padding=True,truncation=True)
print(inputs)
#input = [101, 1045, 2572, 2058, 20192, 2102, 102]
# x = model.get_base_model().forward(inputs["input_ids"])
# print(model())

{'input_ids': tensor([[  101,  1056, 28394,  2102,  3793,  1024,  1030, 11990,  7874,  3215,
         25465,  2115,  8013,  2326,  2038,  2042,  9202,  2076,  1996,  9131,
          2832,  1012,  1045,  2097,  2196,  5309,  1037, 11990,  2153,  1012,
          3830,  1024,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [39]:
print(model(inputs['input_ids'],inputs['token_type_ids'],inputs['attention_mask']))

TypeError: forward() got an unexpected keyword argument 'labels'

In [41]:
bertModel = BertModel.from_pretrained('bert-base-uncased', config=bertconfig)
y = bertModel(inputs["input_ids"])
print(y)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 1.1691e-01, -3.9735e-02, -1.0928e-01,  ..., -3.6141e-01,
           2.6877e-01,  3.3045e-01],
         [ 1.2260e+00, -7.4663e-01,  3.6256e-01,  ...,  1.6007e-01,
           7.5153e-01,  2.7402e-01],
         [ 1.3149e+00, -2.8815e-01,  4.9387e-01,  ..., -6.4974e-01,
          -7.7650e-02, -1.9502e-01],
         ...,
         [ 1.2370e-01, -9.5042e-01,  3.6132e-01,  ..., -1.1498e-01,
          -1.0193e-01, -2.3024e-01],
         [-1.2057e-01,  2.0980e-01, -2.9318e-01,  ..., -3.8923e-01,
           3.7410e-01,  4.9506e-01],
         [ 4.3562e-01,  3.0449e-01,  4.8004e-05,  ...,  6.2245e-02,
          -7.3993e-01, -4.2453e-01]]], grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-0.7736, -0.5322, -0.9692,  0.6489,  0.7484, -0.1469,  0.4637,  0.3117,
         -0.8825, -1.0000, -0.4685,  0.9510,  0.9598,  0.7588,  0.8338, -0.4510,
          0.0442, -0.5428,  0.3221,  0.4124,  0.6666,  1.0000, -0.0673,  0.283

In [43]:
import loralib as lora
# Add a pair of low-rank adaptation matrices with rank r=16
layer = lora.Linear(512, 512, r=16)
lora.mark_only_lora_as_trainable(bertModel)

In [33]:
import torch
print(torch.sub(input=y[1],other=x[1]))

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.